### Research
In well log lithology interpretation, the phrase "building a proper algorithm to map stringers" refers to the process of developing a systematic and accurate method to identify and characterize stringers within the geological formations encountered in a well.

Stringers typically refer to thin, elongated or discontinuous layers or veins of a specific lithology (rock type) within a larger formation. These stringers may have distinct properties or lithologies compared to the surrounding rock units, and identifying and mapping them accurately is important for understanding the geological characteristics of the subsurface.

To "build a proper algorithm" means creating a set of rules or procedures that can be applied to well log data to identify and delineate stringers. This algorithm should leverage the information provided by different well log measurements, such as gamma ray, resistivity, density, neutron porosity, etc., to detect and classify stringers based on their unique signatures.

The algorithm may involve analyzing patterns and anomalies in the well log data, using statistical methods, applying machine learning techniques, or incorporating domain-specific knowledge. The goal is to create a reliable and consistent approach that can be applied across multiple wells or sections of a well to accurately identify and map stringers in the subsurface.

By developing such an algorithm, geoscientists and reservoir engineers can gain insights into the distribution and characteristics of stringers, which can have implications for hydrocarbon reservoir quality, fluid flow behavior, and overall geological modeling and interpretation.

In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import pickle

In [2]:
def drop_columns(data, *args):

    '''
    function used to drop columns.
    args:: 
      data:  dataframe to be operated on
      *args: a list of columns to be dropped from the dataframe

    return: returns a dataframe with the columns dropped
    '''
    
    columns = []
    for _ in args:
        columns.append(_)
        
    data = data.drop(columns, axis=1)
        
    return data


def process(data):

    '''
    function to process dataframe by replacing missing, infinity values with -999

    args:: 
      data:  dataframe to be operated on
    
    returns dataframe with replaced values
    '''
    
    cols = list(data.columns)
    for _ in cols:

        data[_] = np.where(data[_] == np.inf, -999, data[_])
        data[_] = np.where(data[_] == np.nan, -999, data[_])
        data[_] = np.where(data[_] == -np.inf, -999, data[_])
        
    return data

In [3]:
def fill_missing_values(df):

    '''
    function to process dataframe by replacing missing, infinity values with -999

    args:: 
      data:  dataframe to be operated on
    
    returns dataframe with replaced values
    '''
    
    cols = list(df.columns)
    for _ in cols:

        df[_] = np.where(df[_] == np.inf, -999, df[_])
        df[_] = np.where(df[_] == np.nan, -999, df[_])
        df[_] = np.where(df[_] == -np.inf, -999, df[_])
        
    return df


In [38]:
PWD = '/media/Data-B/my_research/Geoscience_FL/data_well_log/'

A = np.load('penalty_matrix.npy')
train = pd.read_csv(PWD + 'train.csv', sep=';')
test = pd.read_csv(PWD + 'test_with_lables.csv')

The augment_features_window function is used to concatenate feature windows in a given dataset. It takes two inputs: X, which is the input feature matrix, and N_neig, representing the number of neighboring windows to consider on each side of the current window. The function performs the following steps:

In [5]:
#Paulo Bestagini's feature augmentation technique from SEG 2016 ML competition
#Link : https://github.com/seg/2016-ml-contest/tree/master/ispl


# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]
 
    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))
 
    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row
 
    return X_aug
 
# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad
 
# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

def score(y_true, y_pred):

    '''
    custom metric used for evaluation
    args:
      y_true: actual prediction
      y_pred: predictions made
    '''

    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [6]:
def show_evaluation(pred, true):

  '''

  function to show model performance and evaluation
  args:
    pred: predicted value(a list)
    true: actual values (a list)

  prints the custom metric performance, accuracy and F1 score of predictions

  '''

  print(f'Default score: {score(true.values, pred)}')
  print(f'Accuracy is: {accuracy_score(true, pred)}')
  print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')

In [7]:
lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']

lithology_numbers = {30000: 0,
                        65030: 1,
                        65000: 2,
                        80000: 3,
                        74000: 4,
                        70000: 5,
                        70032: 6,
                        88000: 7,
                        86000: 8,
                        99000: 9,
                        90000: 10,
                        93000: 11}

labels = lithology.map(lithology_numbers)

In [53]:
def preprocess(df):
        df_well = df.WELL.values
        df_depth = df.DEPTH_MD.values
        
        
        print(f'shape of concatenated dataframe before dropping columns {df.shape}')

        cols = ['SGR', 'DTS', 'RXO', 'ROPA'] #columns to be dropped
        df = drop_columns(df, *cols)
        print(f'shape of dataframe after dropping columns {df.shape}')
        print(f'{cols} were dropped')
        if 'FORCE_2020_LITHOFACIES_LITHOLOGY' in df.columns:
                df.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
                print('FORCE_2020_LITHOFACIES_LITHOLOGY - dropped')

        if 'FORCE_2020_LITHOFACIES_CONFIDENCE' in df.columns:
                df.drop(['FORCE_2020_LITHOFACIES_CONFIDENCE'], axis=1, inplace=True)
                print('FORCE_2020_LITHOFACIES_CONFIDENCE - dropped')

        #Label encoding the GROUP, FORMATION and WELLS features as these improved the performance of the models on validations

        df['GROUP_encoded'] = df['GROUP'].astype('category')
        df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes 
        df['FORMATION_encoded'] = df['FORMATION'].astype('category')
        df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes
        df['WELL_encoded'] = df['WELL'].astype('category')
        df['WELL_encoded'] = df['WELL_encoded'].cat.codes
        print(f'shape of dataframe after label encoding columns {df.shape}')


        #FURTHER PREPRATION TO SPLIT DATAFRAME INTO TRAIN AND TEST DATASETS AFTER PREPRATION
        df = df.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)
        print(df.shape)
        
        df = df.fillna(-999)
        df = process(df)

        print(f'dataframe columns: {df.columns}')

        print(f'Shape of the datasets BEFORE augmentation {df.shape}')
 
        augmented_df, _ = augment_features(pd.DataFrame(df).values, df_well, df_depth)
        

        print(f'Shape of the datasets AFTER augmentation {augmented_df.shape}')
    
        return augmented_df

In [54]:
trainset = preprocess(train)
testset = preprocess(test)

shape of concatenated dataframe before dropping columns (1170511, 29)
shape of dataframe after dropping columns (1170511, 25)
['SGR', 'DTS', 'RXO', 'ROPA'] were dropped
FORCE_2020_LITHOFACIES_LITHOLOGY - dropped
FORCE_2020_LITHOFACIES_CONFIDENCE - dropped
shape of dataframe after label encoding columns (1170511, 26)
(1170511, 23)
dataframe columns: Index(['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DCAL', 'DRHO',
       'MUDWEIGHT', 'RMIC', 'GROUP_encoded', 'FORMATION_encoded',
       'WELL_encoded'],
      dtype='object')
Shape of the datasets BEFORE augmentation (1170511, 23)
Shape of the datasets AFTER augmentation (1170511, 92)
shape of concatenated dataframe before dropping columns (136786, 28)
shape of dataframe after dropping columns (136786, 24)
['SGR', 'DTS', 'RXO', 'ROPA'] were dropped
FORCE_2020_LITHOFACIES_LITHOLOGY - dropped
shape of dataframe after label encoding columns (136786, 26)

In [55]:
split = 10
kf = StratifiedKFold(n_splits=split, shuffle=True)

open_test = np.zeros((len(testset), 12))
      
#100 n-estimators and 10 max-depth
model = XGBClassifier(n_estimators=100, max_depth=10, booster='gbtree',
                            objective='multi:softprob', learning_rate=0.1, random_state=0,
                            subsample=0.9, colsample_bytree=0.9, tree_method='gpu_hist',
                            eval_metric='mlogloss', verbose=2020, reg_lambda=1500)
      
 
i = 1
for (train_index, test_index) in kf.split(pd.DataFrame(trainset), pd.DataFrame(labels)):
        X_train, X_test = pd.DataFrame(trainset).iloc[train_index], pd.DataFrame(trainset).iloc[test_index]
        Y_train, Y_test = pd.DataFrame(labels).iloc[train_index],pd.DataFrame(labels).iloc[test_index]
    
        model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=100)
        prediction = model.predict(X_test)
        print(show_evaluation(prediction, Y_test))
 
        print(f'-----------------------FOLD {i}---------------------')
        i+=1
 
        open_test += model.predict_proba(pd.DataFrame(testset))
      
open_test= pd.DataFrame(open_test/split)
    
open_test = np.array(pd.DataFrame(open_test).idxmax(axis=1))
 
print('---------------CROSS VALIDATION COMPLETE')
print('----------------TEST EVALUATION------------------')

/home/dnlab/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21:00:20] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:2.16206
[99]	validation_0-mlogloss:0.31940
Default score: [-0.27889635]
Accuracy is: 0.8939958309127567
F1 is: 0.897982303870989
None
-----------------------FOLD 1---------------------


/home/dnlab/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21:00:47] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:2.16141
[99]	validation_0-mlogloss:0.31715
Default score: [-0.27780946]
Accuracy is: 0.8946869313376221
F1 is: 0.8985601299747266
None
-----------------------FOLD 2---------------------


/home/dnlab/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[21:03:35] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:2.16210
[99]	validation_0-mlogloss:0.32024
Default score: [-0.27948394]
Accuracy is: 0.8935506744923153
F1 is: 0.8975073903411572
None
-----------------------FOLD 8---------------------


/home/dnlab/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21:04:03] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:2.16243
[99]	validation_0-mlogloss:0.31730
Default score: [-0.27459612]
Accuracy is: 0.8952935045407557
F1 is: 0.8990876336772359
None
-----------------------FOLD 9---------------------


/home/dnlab/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21:04:31] WARNING: ../src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:2.16217
[99]	validation_0-mlogloss:0.31817
Default score: [-0.27663796]
Accuracy is: 0.8949090567359527
F1 is: 0.8988020084805362
None
-----------------------FOLD 10---------------------
---------------CROSS VALIDATION COMPLETE
----------------TEST EVALUATION------------------


In [ ]:
hidden_test = pd.read_csv('/home/dnlab/Data-B/my_research/Geoscience_FL/data_well_log/hidden_test.csv', sep=';')
hidden_test.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-23,1518.280,433906.75,6460000.5,-1493.241821,HORDALAND GP.,Skade Fm.,15.506232,NaN,NaN,...,146.526276,326.451263,-1.993768,0.109706,NaN,NaN,88.968864,NaN,65000,3.0
1,15/9-23,1518.432,433906.75,6460000.5,-1493.393799,HORDALAND GP.,Skade Fm.,18.524611,NaN,NaN,...,147.605148,322.926361,1.024611,-0.006418,NaN,NaN,92.287186,NaN,65000,3.0
2,15/9-23,1518.584,433906.75,6460000.5,-1493.545776,HORDALAND GP.,Skade Fm.,18.855669,NaN,NaN,...,140.783127,325.283142,1.355668,0.022769,NaN,NaN,95.605499,NaN,65000,3.0
3,15/9-23,1518.736,433906.75,6460000.5,-1493.697754,HORDALAND GP.,Skade Fm.,19.163353,NaN,NaN,...,125.159531,334.233185,1.663353,0.024972,NaN,NaN,98.923820,NaN,65000,3.0
4,15/9-23,1518.888,433906.75,6460000.5,-1493.849609,HORDALAND GP.,Skade Fm.,18.489744,NaN,0.849849,...,107.576691,330.952362,0.989743,0.024527,NaN,NaN,102.242142,NaN,65000,3.0


In [56]:
new_test = preprocess(hidden_test)

shape of concatenated dataframe before dropping columns (122397, 29)
shape of dataframe after dropping columns (122397, 25)
['SGR', 'DTS', 'RXO', 'ROPA'] were dropped
FORCE_2020_LITHOFACIES_LITHOLOGY - dropped
FORCE_2020_LITHOFACIES_CONFIDENCE - dropped
shape of dataframe after label encoding columns (122397, 26)
(122397, 23)
dataframe columns: Index(['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DCAL', 'DRHO',
       'MUDWEIGHT', 'RMIC', 'GROUP_encoded', 'FORMATION_encoded',
       'WELL_encoded'],
      dtype='object')
Shape of the datasets BEFORE augmentation (122397, 23)
Shape of the datasets AFTER augmentation (122397, 92)


In [59]:
prediction = model.predict(new_test)
prediction

array([2, 2, 2, ..., 2, 2, 2])

In [60]:
lithology_labels = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}

In [65]:
lithology_dict = {value: lithology_labels[key] for key, value in lithology_numbers.items()}
lithology_dict


{0: 'Sandstone',
 1: 'Sandstone/Shale',
 2: 'Shale',
 3: 'Marl',
 4: 'Dolomite',
 5: 'Limestone',
 6: 'Chalk',
 7: 'Halite',
 8: 'Anhydrite',
 9: 'Tuff',
 10: 'Coal',
 11: 'Basement'}

In [66]:
prediction_labels = [lithology_dict[pred] for pred in prediction]
prediction_labels

['Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
 'Shale',
